In [2]:
#the data is obtained from the Victoria building authority. https://www.vba.vic.gov.au/about/data
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyxlsb import open_workbook as open_xlsb
from pyxlsb import convert_date
import openpyxl
from openpyxl import load_workbook
import os
from opencage.geocoder import OpenCageGeocode
from opencage.geocoder import InvalidInputError, RateLimitExceededError, UnknownError

In [3]:
def load_extract_data(filepath):
    
    df = []
    with open_xlsb(filepath) as wb:
        with wb.get_sheet('Sheet1') as sheet:
            for row in sheet.rows():
                df.append([item.v for item in row])
                
    #first row are the column names         
    df = DataFrame(df[1:], columns=df[0])            
    
    df.columns = [col.capitalize() for col in df.columns]
    
    #converting the dates from numbers into date format
    df['Permit_date'] = df['Permit_date'].apply(lambda x: convert_date(x) if pd.notnull(x) else None)
    df['Permit_app_date'] = df['Permit_app_date'].apply(lambda x: convert_date(x) if pd.notnull(x) else None)
    
    #dropping out the columns not of interest
    cols_of_int = ['Permit_stage_number','Permit_date','Permit_app_date','Calculated_levy_amount','Site_street','Site_suburb','Region','Sub_region','Sub_region1','Basis_zone','Basis_now','Basis_bca']
    drop_cols = [c for c in df.columns if c not in cols_of_int]
    df.drop(columns=drop_cols, inplace=True)
    
    #converting the data types
    df=df.astype({'Permit_stage_number':'int32', 'Basis_now':'category','Basis_bca':'category','Basis_zone':'category'})
    
    #filtering out the data related to metropolitan region and making a copy of it
    metro_df = df[df['Region']=='Metropolitan'].copy()
    
    #building class simplified
    metro_df.loc[:,'_Building_class'] = metro_df['Basis_bca'].apply(lambda x: '10' if x[:2] == '10' else x[:1])
    metro_df['_Building_class'] = metro_df['_Building_class'].astype('category')
    
    return metro_df


class geocode:
    
    def __init__(self):
        #create a geocode connection with your key
        self.key = '1dead48e815c47049178cdd313a5687c'
        self.geocoder = OpenCageGeocode(self.key)
        
    def geocode_fetch(self, location_list):
        geo_df = DataFrame(columns=['latitude','longitude'])
        
        for loc in location_list:
            query = loc+',Victoria,Australia'
            try:
                results = self.geocoder.geocode(query)
                if(results) and len(results):
                    geo_df.loc[loc, 'latitude']= results[0]['geometry']['lat']
                    geo_df.loc[loc, 'longitude']= results[0]['geometry']['lng']   
            except RateLimitExceededError as re:
                print(re)   
            except UnknownError as ue:
                print(ue)
            except Exception as e:
                print(e)
        return geo_df
        

In [11]:
def save_to_excel(output_file, data, sheets):    
    writer = pd.ExcelWriter(output_file, engine = 'openpyxl')
    
    if(os.path.exists(output_file)):
        writer.book = load_workbook(output_file)
        writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    
    for (dframe,sheet) in zip(data,sheets):
        if sheet in writer.sheets.keys():
            reader = pd.read_excel(output_file, sheet_name=sheet)
            dframe.to_excel(writer, sheet,header = False, startrow =len(reader)+1)
        else:
            dframe.to_excel(writer, sheet)
                   
    writer.close()        
    return        

In [5]:
folder = './VBA-permits'
output_file =  folder+'/output/VBA_Metro_permits.xlsx'

for file in os.listdir(folder):
    
    filepath = folder+'/'+file
    #get the file name without extension and split to get the last element - year
    print('filename - {}'.format(file))
    print('input filepath - {}, o/p file path - {}'.format(filepath, output_file))
    year = os.path.splitext(file)[0].split('-')[-1]

    metro_data = load_extract_data(filepath)
    geocode_data = DataFrame()
    geoencoder = geocode()
    
    if(os.path.exists(output_file)):
        geocode_data = pd.read_excel(output_file, sheet_name='geodata')
        
    #get the unique list of suburbs which does not have the geocode data fetched
    new_suburb_list = np.setdiff1d(metro_data['Site_suburb'].tolist(), geocode_data.index.tolist())
    
    #split the new suburb list into 10 groups rather than a big batch
    print('Going to read geocode for {} suburbs - sit back '.format(len(new_suburb_list)))
    for list_p in np.array_split(new_suburb_list,10):
        try:
            geocode_data = geocode_data.append(geoencoder.geocode_fetch(list_p))    
        except:
            print('shape of geocode_data:{}'.format(geocode_data.shape))
     
    
    save_to_excel(output_file, [metro_data, geocode_data.loc[new_suburb_list]], [year, 'geodata']) 
    
    


filename - VBA-DataVic-Building-Permits-2018.xlsb
input filepath - ./VBA-permits/VBA-DataVic-Building-Permits-2018.xlsb, o/p file path - ./VBA-permits/output/VBA_Metro_permits.xlsx
Going to read geocode for 633 suburbs - sit back 


AttributeError: module 'os' has no attribute 'file'

In [13]:
save_to_excel(output_file, [metro_data, geocode_data.loc[new_suburb_list]], [year, 'geodata'])